In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler #標準化のコード

# データの読み込み
# ファイル名を実際の訓練・テストデータファイル名に置き換えてください
train_data_LR = pd.read_csv('selected_train_set.csv')
test_data_LR = pd.read_csv('selected_test_set.csv')
#index_col=['id']

# 特徴量（X）と目標変数（y）の設定
#11/1のミーティングで決まった項目も抜いて機械学習

X_train_LR = train_data_LR.drop(columns=['PassengerId','Survived'])  # 'target_column'を目標変数の列名に置き換えてください
y_train_LR = train_data_LR['Survived']
X_test_LR = test_data_LR.drop(columns=['PassengerId','Survived'])
y_test_LR = test_data_LR['Survived']

# 特徴量の数を確認
num_features_LR = X_train_LR.shape[1]
print("特徴量の数:", num_features_LR)

# 標準化
scaler = StandardScaler()
X_train_standard_LR = scaler.fit_transform(X_train_LR)
X_test_standard_LR = scaler.transform(X_test_LR)
# 10回ループのためのリストを初期化
accuracies_LR = []  # 精度を格納するリスト
# 重要度を格納するリスト
importances_list_LR = []

predicted_probabilities_LR = []  # 予測確率を格納するリスト

random_state = 42
# 10回ループでモデルを作成
for i in range(10):
    # sagaソルバーとElastic Netペナルティを使用したロジスティック回帰モデルを定義
    model_LR = LogisticRegression(
    solver='saga',           # 小さなデータセットに最適なソルバーで正則化対応
    penalty='elasticnet',    # Elastic Netで特徴選択と正則化を両立
    l1_ratio=0.5,            # L1とL2のバランス（0.5は出発点として適切）
    C=1.0,                   # 正則化の強度（調整やチューニングも可能）
    max_iter=10000,            # 収束を確実にするために最大イテレーションを増加   #5000だと精度0.50、
    random_state=random_state  # Set random state for reproducibility
)
    # ロジスティック回帰モデルの作成
    #model = LogisticRegression(max_iter=100, random_state=i)  # max_iterで反復回数を調整 #100で0.50,200で0.44
    #model = LogisticRegression(solver='liblinear')#0.44だった

    # モデルの訓練
    model_LR.fit(X_train_standard_LR, y_train_LR)

# 回帰係数を取得（11/1）
    coefficients_LR = model_LR.coef_[0]  # 係数は配列として返されるので、最初の要素を取得
    
    # テストデータでの予測
    y_pred_LR = model_LR.predict(X_test_standard_LR)

    # 特徴量の重要度を取得し、リストに追加 11/1
    importances_list_LR.append(np.abs(coefficients_LR))  # 絶対値を取って追加

    # テストデータで予測確率を取得し、リストに追加
    y_scores_LR = model_LR.predict_proba(X_test_standard_LR)[:, 1]
    predicted_probabilities_LR.append(y_scores_LR)
    
    # モデルの精度を計算 このモデルの適応度みたいなもの
    accuracy_LR = accuracy_score(y_test_LR, y_pred_LR)
    accuracies_LR.append(accuracy_LR)
# 平均精度を計算
average_accuracy_LR = np.mean(accuracies_LR)
print(f'Average Accuracy over 10 runs: {average_accuracy_LR:.2f}')

from sklearn.model_selection import cross_val_score   #一般化敵には、max_iterは10000の方がよい。収束している
accuracies_LR = cross_val_score(model_LR, X_train_standard_LR, y_train_LR, cv=10)
print(f'Average Accuracy over 10 folds: {np.mean(accuracies_LR):.2f}')

# 特徴量の重要度の平均を計算
mean_importances_LR = np.mean(importances_list_LR, axis=0)
# 特徴量の名前と重要度の平均をデータフレームにまとめる
feature_importance_df_LR = pd.DataFrame({
    'Feature': X_train_LR.columns,
    'Importance': mean_importances_LR
})

# 重要度でソート
feature_importance_df_LR = feature_importance_df_LR.sort_values(by='Importance', ascending=False)
# 上位13項目を抽出
top_13_importance_df_LR = feature_importance_df_LR.head(13)

# グラフの作成
plt.figure(figsize=(20, 16))
plt.barh(top_13_importance_df_LR['Feature'], top_13_importance_df_LR['Importance'], color='skyblue')
plt.xlabel('Importance')
plt.title('Top 13 Feature Importances ')
plt.gca().invert_yaxis()
plt.show()

from sklearn.metrics import confusion_matrix, roc_curve, auc

y_scores_LR = model_LR.predict_proba(X_test_standard_LR)[:, 1]  # クラス1の確率を取得
#print(y_scores)

# ROC計算用のしきい値を決定（スコアの一意な値を使用）
thresholds_LR = np.sort(np.unique(y_scores_LR))

# 感度（TPR）と特異度を計算
tpr_list_LR = []
specificity_list_LR = []

for threshold in thresholds_LR:
    # スコアをしきい値で2値化
    y_pred_LR = (y_scores_LR >= threshold).astype(int)
    
    # 混同行列を取得
    tn_LR, fp_LR, fn_LR, tp_LR = confusion_matrix(y_test_LR, y_pred_LR).ravel()
    
    # 感度と特異度を計算
    tpr_LR = tp_LR / (tp_LR + fn_LR) if (tp_LR + fn_LR) > 0 else 0  # 感度
    specificity_LR = tn_LR / (tn_LR + fp_LR) if (tn_LR + fp_LR) > 0 else 0  # 特異度
    
    # リストに保存
    tpr_list_LR.append(tpr_LR)
    specificity_list_LR.append(specificity_LR)

# AUCの計算
roc_auc_LR = auc(np.array(specificity_list_LR), tpr_list_LR)

# ROC曲線を描画
plt.figure(figsize=(8, 6))
plt.plot( np.array(specificity_list_LR), tpr_list_LR, label=f"ROC Curve (AUC = {roc_auc_LR:.2f})", color="blue")
plt.plot([1, 0], [0, 1], linestyle="--", color="red", label="Random Guess")

# グラフの装飾
plt.xlim([1.0, 0.0])  # X軸を1から0に反転
plt.ylim([0.0, 1.0])
plt.xlabel("Specificity (1 - FPR)")
plt.ylabel("Sensitivity (TPR)")
plt.title("ROC Curve with TP, FP, TN, FN")
plt.legend(loc="lower left")
plt.grid()
plt.show()

from sklearn.metrics import confusion_matrix
# 混同行列の計算
tn_LR, fp_LR, fn_LR, tp_LR = confusion_matrix(y_test_LR, y_pred_LR).ravel()

# 感度と特異度の計算
sensitivity_LR = tp_LR / (tp_LR + fn_LR)  # 真陽性率
specificity_LR = tn_LR / (tn_LR + fp_LR)  # 真陰性率

print(f"Sensitivity: {sensitivity_LR:.4f}")
print(f"Specificity: {specificity_LR:.4f}")

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# 混同行列の計算
cm_LR = confusion_matrix(y_test_LR, y_pred_LR)

# 混同行列の表示
disp_LR = ConfusionMatrixDisplay(confusion_matrix=cm_LR, display_labels=["Negative", "Positive"])
disp_LR.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# 混同行列の要素を使って感度と特異度を計算
tn_LR, fp_LR, fn_LR, tp_LR = cm_LR.ravel()
sensitivity_LR = tp_LR / (tp_LR + fn_LR)  # 感度
specificity_LR = tn_LR / (tn_LR + fp_LR)  # 特異度

print(f"Sensitivity: {sensitivity_LR:.2f}")
print(f"Specificity: {specificity_LR:.2f}")

In [ ]:
from sklearn.model_selection import learning_curve
# ロジスティック回帰モデルの定義
#model = LogisticRegression(random_state=42, max_iter=1000)

# 学習曲線の計算
train_sizes, train_scores, test_scores = learning_curve(
    model_LR, X_train_standard_LR, y_train_LR, cv=5, n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 10)
)

# 各スコアの平均値と標準偏差を計算
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

# 学習曲線のプロット
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_scores_mean, color='blue', label='Training Score')
plt.plot(train_sizes, test_scores_mean, color='green', label='Cross-validation Score')

# 標準偏差の影響をプロット
plt.fill_between(train_sizes, 
                 train_scores_mean - train_scores_std, 
                 train_scores_mean + train_scores_std, 
                 color='blue', alpha=0.2)
plt.fill_between(train_sizes, 
                 test_scores_mean - test_scores_std, 
                 test_scores_mean + test_scores_std, 
                 color='green', alpha=0.2)

plt.title('Learning Curve for Logistic Regression')
plt.xlabel('Training Set Size')
plt.ylabel('Score')
plt.legend(loc='best')
plt.grid()
plt.show()